### Загрузка библиотек
---
Эмбэддинг модель и создание/сохранение базы данных
```py
lib.multi_gpu_embeddings
```
- **MultiGPUHuggingFaceEmbeddings**
- _create_empty_faiss_index_
- _create_or_load_faiss_index_
- _save_faiss_index_
---
Большая языковая модель для генерации ответов
```py
lib.llm
```
- **LLM**
---
Занесение документов Docx в базу данных
```py
lib.docx_to_lang
```
- _docx2faiss_
- _iter_docx_data_

In [1]:
# Библиотеки
import torch
from langchain.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

# Локальные файлы
from lib.multi_gpu_embeddings import *
from lib.llm import LLM
from lib.docx_to_lang import docx2faiss

In [2]:
# Загрузка модели поиска данных
embedding_model = MultiGPUHuggingFaceEmbeddings(
		model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
		half=True,
		model_kwargs={"device": "cuda" if torch.cuda.is_available() else "mps" },
		encode_kwargs={"normalize_embeddings": True},
		multi_process=False
	)

In [ ]:
# Загрузка БД википедии
vectorstore = FAISS.load_local("fiass", embedding_model, allow_dangerous_deserialization = True, distance_strategy = DistanceStrategy.COSINE)

### Пример загрузки Docx в базу данных

1) Загрузи NLP модель

```py
nlp = spacy.load('<model>')
```

2) Загрузи/создай индекс FAISS

```py
vs = FAISS(...)
```

3) Загрузи свои отчёты в функцию:

```py
docx2faiss([
	'doc1.docx',
	'doc2.docx',
	'doc3.docx',
], vs, nlp)
```

In [4]:
# Загрузка NLP модели

import spacy

nlp = spacy.load('ru_core_news_sm')

In [6]:
# Занесение данных в новую БД

vectorstore = create_empty_faiss_index(embedding_model)

docx2faiss(["documents/Бирюлево_Быкова_ред.docx","documents/отчет по выполнению муниципального задания во 2 квартале 2020 г.квартал.docx"],vectorstore,nlp)

save_faiss_index("fiass/", vectorstore)

# Поиск наиболее похожих k результатов по косинусному расстоянию.
Здесь применяется алгоритм KNN

In [39]:
# Теперь вы можете сделать запрос на похожие документы
query = "Заключенные в гулагах"
docs = vectorstore.similarity_search(query, k=5)  # k - сколько результатов получить

for doc in docs:
	print("Содержимое документа:", doc.page_content)
	print("Метаданные:", doc.metadata)
	print("------")

Содержимое документа: Первый (временный) снежный покров образуется обычно в последних числах октября — первых числах ноября.
Как правило, он сразу же сходит, поскольку почва ещё недостаточно остыла, а также в это время высока активность западного переноса, что позволяет среднесуточной температуре оставаться положительной.
В ноябре резко преобладает пасмурная погода (из-за высокой влажности атмосферы), хотя во время прохождения антициклонов иногда устанавливается очень тёплая для ноября погода (до +10 °C и выше днём).
В отдельные годы первый снег сопровождается гололёдом.
Так было, в частности, 30—31 октября 2012 года, когда на Москву обрушился сильнейший ледяной дождь.
Ледяной переохлаждённый дождь, идущий при отрицательной температуре воздуха, был отмечен в Москве и 3 ноября 2014 года.
Среднесуточная температура опускается ниже 0 °C в середине ноября, а устойчивый снежный покров и общий зимний характер погоды устанавливается в конце ноября.
Правда, в последние 

In [3]:
query = "Государственный долг США"
docs = vectorstore.similarity_search(query, k=5)

for doc in docs:
    print("Содержимое документа:", doc.page_content)
    print("Метаданные:", doc.metadata)
    print("------")

Содержимое документа: Государственный долг США
15 ноября 2024 года сумма государственного долга США впервые в истории превысила $36 трлн.
История
Историческая динамика долга
Госдолг США по годам на 31 декабря
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
Год | Госдолг США, млрд $ | ВВП, млрд $ | Доля госдолга от ВВП, %
1910 | 2,7 | 33,2 | 8,1
1920 | 26,0 | 89,0 | 29,2
1930 | 16,2 | 98,4 | 16,5
1940 | 50,7 | 98,2 | 51,6
1950 | 256,9 | 279,0 | 94,0
1960 | 290,5 | 535,1 | 54,3
1970 | 380,9 | 1049,1 | 36,3
1980 | 909,0 | 2796,8 | 32,5
1990 | 3206,3 | 5914,6 | 54,2
2000 | 5628,7 | 10148,2 | 55,5
2010 | 13 528,7 | 14 798,5 | 91,4
2017 | 20 164,0 | 19 362,1 | 104,1
2020 | 26 000,0 | 20 600,0 | 126,2
2022 | 31 000,0 | 23 000,0 | 134,8
2024 | 35 000,0 | 26 940,0 | 129,8
Метаданные: {'uuid': 'wiki-163538-Государственный_долг_США-72796', 'header': ['Год', 'Госдолг США, млрд $', 'ВВП, млрд $', 'Доля госдолга от ВВП, %']}
------
Содержимое документа: Экономика США
В 1791 году до

In [15]:
query = "Достижения качества работ"
docs = vectorstore.similarity_search(query, k=5)

for doc in docs:
    print("Содержимое документа:", doc.page_content)
    print("Метаданные:", doc.metadata)
    print("------")

Содержимое документа: отчет по выполнению муниципального задания во 2 квартале 2020 г.квартал
2.2.
Категории потребителей работы: в интересах общества
2.3.
Сведения о фактическом достижении показателей, характеризующих объем и (или) качество работы:
2.4.
Сведения о фактическом достижении показателей, характеризующих качество работ:

№ п/п | Показатели качества работы | Показатели качества работы | Показатели качества работы | Показатели качества работы | Показатели качества работы | Показатели качества работы | Показатели качества работы
№ п/п | наименование показателя | единица измерения | утверждено в муниципальном задании на год | исполнено на отчетную дату | допустимое (возможное) отклонение | отклонение, превышающее допустимое (возможное) значение | причина отклонения
1 | 2 | 3 | 4 | 5 | 6 | 7 | 8
1. | Количество документов | Ед. | 13200 | 13791 | 5% |  | 
 |  |  |  |  |  |  | 
 |  |  |  |  |  |  |
Метаданные: {'uuid': 'wiki-1-отчет_по_выполнению_муниципального_задания_во_2_кварта

### Загружаем языковую модель

Здесь была использована русскоязычная модель [t-tech/T-lite-it-1.0](https://huggingface.co/t-tech/T-lite-it-1.0)

In [4]:
llm = LLM("t-tech/T-lite-it-1.0")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
llm.answer(doc.page_content,"Вся информация о госдолге США на 2024 год")

'Информация о государственном долге США на 2024 год не содержится в данном контексте. Для получения актуальных данных о государственном долге США рекомендуется обратиться к официальным источникам, таким как Министерство финансов США или Федеральная резервная система. (Согласно моим знаниям на 2023 год, конкретные цифры могут изменяться.)'